## Load in data

In [41]:
%load_ext autoreload
%autoreload 2
from ais_dataloader import *
import ipywidgets as widgets
from IPython.display import display
from plotting_utils import *



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cpu


In [42]:
date_range = pd.date_range(start='2024-01-01', end='2024-01-01', freq='D')
gp_regression_dataset = AISTrajectoryRegressionDataset(date_range, device)


Loading cached dataframe from data/processed/processed_AIS_df_2024_01_01_2024_01_01.pkl


Scaling trajectories for each MMSI: 100%|██████████| 3453/3453 [00:01<00:00, 2771.07it/s]



===== Dataset Statistics =====
Total number of AIS messages: 2128288
Number of unique MMSIs: 3453
Date range: 2024-01-01 00:00:00 to 2024-01-01 23:59:59


## Fit GP Models

In [ ]:
import torch
import gpytorch
from multioutput_gp import *

num_trajectories = 10
models = {}
likelihoods = {}
losses = {}

for idx in range(num_trajectories):
    # mmsi, times, state_trajectory = gp_regression_dataset[idx]
    mmsi, times, state_trajectory = gp_regression_dataset[idx]
    
    print(f"\nFitting GP for trajectory {idx+1}/{num_trajectories} for MMSI {mmsi}")
    
    X = times.detach().unsqueeze(1).to(device)
    Y = state_trajectory.detach().to(device)
    
    num_outputs = Y.shape[1]
    
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_outputs, noise_prior=gpytorch.priors.NormalPrior(loc=0.25, scale=0.25)).to(device)
    model = MultiOutputExactGPModel(X, Y, likelihood, num_outputs=num_outputs).to(device)

    with gpytorch.settings.cholesky_jitter(1e-3):
        loss, model, likelihood = train_model(model, likelihood, X, Y, num_epochs=250, lr=0.1)

    print(f"Loss: {loss.item()}")
    # models.append(model)
    # likelihoods.append(likelihood)
    # losses.append(loss.item())
    models[mmsi] = model
    likelihoods[mmsi] = likelihood
    losses[mmsi] = loss.item()

<>:14: SyntaxWarning: invalid escape sequence '\F'
<>:14: SyntaxWarning: invalid escape sequence '\F'
/var/folders/vw/3pp1qrb97m97jj_776599z6c0000gn/T/ipykernel_75181/3892422781.py:14: SyntaxWarning: invalid escape sequence '\F'
  print(f"\Fitting GP for trajectory {idx+1}/{num_trajectories} for MMSI {mmsi}")


\Fitting GP for trajectory 1/10 for MMSI 3660489


GP Training Progress:   0%|          | 0/250 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/gp/lib/python3.13/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-03 to the diagonal
  warnings.warn(
GP Training Progress: 100%|██████████| 250/250 [00:00<00:00, 267.97it/s]


Loss: 0.8208361268043518
\Fitting GP for trajectory 2/10 for MMSI 203661016


GP Training Progress:  82%|████████▏ | 206/250 [00:09<00:02, 17.68it/s]

In [ ]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.tolist()}')
    
print()
print(model.covar_module.data_covar_module.kernels[0].lengthscale.item())

Parameter name: likelihood.raw_task_noises                 value = [0.4403277635574341, 0.4598369598388672, 0.4579230844974518, -5.29512882232666, -5.297037124633789, -5.295773983001709]
Parameter name: likelihood.raw_noise                       value = [-5.250217914581299]
Parameter name: mean_module.base_means.0.raw_constant      value = 0.21668961644172668
Parameter name: mean_module.base_means.1.raw_constant      value = 0.07388921827077866
Parameter name: mean_module.base_means.2.raw_constant      value = -0.02350754663348198
Parameter name: mean_module.base_means.3.raw_constant      value = -0.000836276332847774
Parameter name: mean_module.base_means.4.raw_constant      value = 0.000478812784422189
Parameter name: mean_module.base_means.5.raw_constant      value = -2.2941880160942674e-05
Parameter name: covar_module.task_covar_module.covar_factor value = [[-1.114229440689087], [-0.316337913274765], [0.4232384264469147], [-0.0008093673968687654], [7.027894753264263e-06], [-8.42002

In [ ]:
# Set the model and likelihood to evaluation mode
model.eval()
likelihood.eval()

# Generate test inputs (e.g., evenly spaced time points)
test_times = torch.linspace(times.min(), times.max(), 200).unsqueeze(1).to(device)

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cholesky_jitter(1e-3):
    predictions = likelihood(model(test_times))
    mean = predictions.mean
    lower, upper = predictions.confidence_region()



/opt/homebrew/Caskroom/miniconda/base/envs/gp/lib/python3.13/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-03 to the diagonal
  warnings.warn(


### Plot GP Solution

In [ ]:
# model.eval()
# likelihood.eval()
# from plotting_utils import *

# for mmsi in models:
#     model = models[mmsi]
#     likelihood = likelihoods[mmsi]
#     # Get the corresponding data for this MMSI
#     # If you want to use the same train/test split as before:
#     times, state_trajectory = None, None
#     for entry in gp_regression_dataset:
#         if entry[0] == mmsi:
#             _, times, state_trajectory = entry
#             break
#     if times is None:
#         continue  # skip if MMSI not found

#     train_X = times.clone().detach().unsqueeze(1).cpu()
#     train_Y = state_trajectory.clone().detach().cpu()

#     test_X = torch.linspace(times.min(), times.max(), 500).unsqueeze(1).to(device)
    
#     test_Y = eval_model(model, likelihood, test_X)

#     plot_gp(train_X, train_Y, test_X, test_Y)
#     plot_single_ship_path(mmsi, times, state_trajectory)


In [ ]:
# import ipywidgets as widgets
# from IPython.display import display, clear_output
# from plotting_utils import *


# def plot_for_mmsi(selected_mmsi):
#     clear_output(wait=True)
#     model = models[selected_mmsi]
#     likelihood = likelihoods[selected_mmsi]
#     # Get the corresponding data for this MMSI
#     times, state_trajectory = None, None
#     for entry in gp_regression_dataset:
#         if entry[0] == selected_mmsi:
#             _, times, state_trajectory = entry
#             break
#     if times is None:
#         print("No data for MMSI:", selected_mmsi)
#         return

#     train_X = times.clone().detach().unsqueeze(1).cpu()
#     train_Y = state_trajectory.clone().detach().cpu()
#     test_X = torch.linspace(times.min(), times.max(), 500).unsqueeze(1).to(device)
#     test_Y = eval_model(model, likelihood, test_X)

#     plot_gp(train_X, train_Y, test_X, test_Y)
#     plot_single_ship_path(selected_mmsi, times, state_trajectory)

    
# mmsi_dropdown = widgets.Dropdown(
#     options=list(models.keys()),
#     description='MMSI:',
#     disabled=False,
# )

# widgets.interact(plot_for_mmsi, selected_mmsi=mmsi_dropdown)
    
    
    

In [ ]:
print(pd.unique(gp_regression_dataset.df['MMSI'].values))
gp_regression_dataset.get_vessel_group_by_mmsi(3660489)

[367669550 367118980 636018568 ... 367619000 309108000 368926390]


'Other'

## Create the kernel param to ship mmsi dataset


In [ ]:
from gp_kernel_ship_classification_dataset import *
kernel_classification_dataset = GPKernelShipClassificationDataset(gp_regression_dataset, models, device)
kernel_classification_dataset.get_unique_group_ids()

[4]

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from gp_kernel_ship_classification_network import KernelShipClassificationNetwork

# Train params
batch_size = 32
num_epochs = 20
learning_rate = 1e-3

train_loader = DataLoader(kernel_classification_dataset, batch_size=batch_size, shuffle=True)

num_classes = max(kernel_classification_dataset.get_unique_group_ids())

model = KernelShipClassificationNetwork(input_dim=2, num_classes=num_classes+1).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for mmsi, kernel_params, group_id in train_loader:
        kernel_params = kernel_params.to(device)
        group_id = group_id.to(device)  # should be LongTensor for CrossEntropyLoss

        optimizer.zero_grad()
        outputs = model(kernel_params)
        loss = criterion(outputs, group_id)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * kernel_params.size(0)
        _, predicted = outputs.max(1)
        correct += (predicted == group_id).sum().item()
        total += group_id.size(0)

    epoch_loss = running_loss / total
    accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Accuracy: {accuracy:.4f}")

Epoch 1/20 | Loss: 1.8975 | Accuracy: 0.0000
Epoch 2/20 | Loss: 1.8662 | Accuracy: 0.0000
Epoch 3/20 | Loss: 1.8351 | Accuracy: 0.0000
Epoch 4/20 | Loss: 1.8043 | Accuracy: 0.0000
Epoch 5/20 | Loss: 1.7737 | Accuracy: 0.0000
Epoch 6/20 | Loss: 1.7433 | Accuracy: 0.0000
Epoch 7/20 | Loss: 1.7132 | Accuracy: 0.0000
Epoch 8/20 | Loss: 1.6831 | Accuracy: 0.0000
Epoch 9/20 | Loss: 1.6535 | Accuracy: 0.0000
Epoch 10/20 | Loss: 1.6252 | Accuracy: 0.0000
Epoch 11/20 | Loss: 1.5977 | Accuracy: 0.0000
Epoch 12/20 | Loss: 1.5703 | Accuracy: 0.0000
Epoch 13/20 | Loss: 1.5429 | Accuracy: 0.0000
Epoch 14/20 | Loss: 1.5155 | Accuracy: 0.0000
Epoch 15/20 | Loss: 1.4881 | Accuracy: 0.5000
Epoch 16/20 | Loss: 1.4609 | Accuracy: 1.0000
Epoch 17/20 | Loss: 1.4336 | Accuracy: 1.0000
Epoch 18/20 | Loss: 1.4063 | Accuracy: 1.0000
Epoch 19/20 | Loss: 1.3790 | Accuracy: 1.0000
Epoch 20/20 | Loss: 1.3516 | Accuracy: 1.0000
